<a href="https://colab.research.google.com/github/rdstevens27/student/blob/master/listshift_newbegin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
# Pushing 1st range data from 1st Nov 2020 to 31st jan 2020 final code   1st version

import requests
import json
from datetime import datetime
from datetime import timedelta
import os
import hashlib
import datetime as dt
import pandas as pd
from copy import deepcopy
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
from pandas import DataFrame

project_id = 'hnmc-data-science'
dataset_id = 'wheniwork'
table_id = 'listShifts'

# TODO(developer): Set key_path to the path to the service account key
#                  file.

key_path = "/hnmc-data-science-1e5996a1f37c.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(credentials=credentials, project=project_id)
dataset = client.dataset(dataset_id)
table_ref = dataset.table(table_id)

time_schema = [
    bigquery.SchemaField("current_time", "TIMESTAMP", mode="REQUIRED")]

listShift_schema = [
       bigquery.SchemaField("id", "INT64", mode="REQUIRED"),
       bigquery.SchemaField("account_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("user_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("location_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("position_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("site_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("start_time", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("end_time", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("break_time", "FLOAT64", mode="NULLABLE"),
       bigquery.SchemaField("color", "STRING", mode="NULLABLE"),
       bigquery.SchemaField("notes", "STRING", mode="NULLABLE"),
       bigquery.SchemaField("alerted", "BOOL", mode="NULLABLE"),
      # bigquery.SchemaField("linked_users", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("shiftchain_key", "STRING", mode="NULLABLE"),
       bigquery.SchemaField("published", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("published_date", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("notified_at", "STRING", mode="NULLABLE"),
       bigquery.SchemaField("instances", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("created_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("updated_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("acknowledged", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("acknowledged_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("creator_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("is_open", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("actionable", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("block_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("requires_openshift_approval", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("openshift_approval_request_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("is_shared", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("UNIQUE_KEY", "STRING", mode="REQUIRED"),
       bigquery.SchemaField("VERSION_DATE", "TIMESTAMP", mode="NULLABLE")
       ]

def main():

   df = get_all_data()
   load_to_bigquery(df)

  #  last_query_time_df = fetch_data()
  #  get_all_data(last_query_time_df)

# def fetch_data():

#     sql_source = """ SELECT * FROM `hnmc-data-science.wheniwork.currentShiftTime` """             

#     df_source = client.query(sql_source).to_dataframe()

#     return df_source

def get_all_data():

    r = requests.post('https://api.login.wheniwork.com/login', json={"email":"rdoshi@holyname.org","password":"wheniwork123"})
    r.status_code
    data= r.json()

    token=data["token"]
    #print(token)
    endpoint = "https://api.wheniwork.com/2/shifts"

    headers = {"Authorization": token}

    all_data = requests.get(endpoint, params={'start': '01 Nov 2020', 'end': '31 Jan 2021', 'include_allopen': True}, headers=headers).json()
    print(all_data)

    id_ofall_data = []
    for i in range(0, len(all_data['shifts'])):

      id_ofall_data.append(all_data['shifts'][i]['id'])

    print(id_ofall_data)

    new_list = []
    for i in range(0, len(all_data['shifts'])):

      if all_data['shifts'][i]['id'] in id_ofall_data: 
        
        #print(all_data['shifts'][i])
        new_list.append(all_data['shifts'][i])

    # convert list of dictionary to dataframe

    #print(new_list)
    df = pd.DataFrame(new_list)

    df.to_csv("before_time_conv.csv", index = False)
    df['start_time'] = df['start_time'].astype(str).str[5:-6]
    df['end_time'] = df['end_time'].astype(str).str[5:-6]
    df['published_date'] = df['published_date'].astype(str).str[5:-6]
    df['notified_at'] = df['notified_at'].astype(str).str[5:-6]
    df['created_at'] = df['created_at'].astype(str).str[5:-6]
    df['updated_at'] = df['updated_at'].astype(str).str[5:-6]
    df['acknowledged_at'] = df['acknowledged_at'].astype(str).str[5:-6]
    df.to_csv("after_time_conv.csv", index = False)
    # print(df)
    df['start_time'] =  pd.to_datetime(df['start_time'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['end_time'] =  pd.to_datetime(df['end_time'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['published_date'] =  pd.to_datetime(df['published_date'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['notified_at'] =  pd.to_datetime(df['notified_at'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['created_at'] =  pd.to_datetime(df['created_at'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['updated_at'] =  pd.to_datetime(df['updated_at'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['acknowledged_at'] =  pd.to_datetime(df['acknowledged_at'], format='%d %b %Y %H:%M:%S', errors='ignore')
   
    print(df['updated_at'].dtypes)
    #generating Hash key
    gene_hash = {
        'listShifts': ['id']
    }

    key_combination = gene_hash.get('listShifts')
    
    df['UNIQUE_KEY'] = df[key_combination].apply(lambda row: '_'.join(row.values.astype(
        str)), axis=1).astype(str).str.encode('utf-8').apply(lambda x: (hashlib.sha512(x).hexdigest().upper()))

    df['VERSION_DATE'] = datetime.now()
    print(df.info())
    df.to_csv("linked_user.csv", index = False)
    df.drop('linked_users', axis=1, inplace=True)
    print(df.info())
    df.to_csv("linked_user_rm.csv", index = False)
    # # check for updated data from API call
    # print(df['updated_at'])
    # print(last_query_time_df['current_time'][0])
    # last_query_time_df['current_time'] = last_query_time_df['current_time'].astype(str).str[:-6]
    # last_query_time_df['current_time'] =  pd.to_datetime(last_query_time_df['current_time'], format='%d %b %Y %H:%M:%S', errors='ignore')
    # print(last_query_time_df['current_time'][0])
    # new_data_df = df[df['updated_at'] >= (last_query_time_df['current_time'][0])]
    # load_to_bigquery(new_data_df)

    # Assign current time to our new currentShiftTime table

    current_shift_time = datetime.now() - timedelta(hours=5)
    current_time = {'current_time': current_shift_time.strftime("%Y-%m-%d %H:%M:%S")}
    
    current_time_df = pd.DataFrame(current_time, columns = ['current_time'], index=[0])

    current_time_df['current_time'] =  pd.to_datetime(current_time_df['current_time'], format='%Y-%m-%d %H:%M:%S')

    load_to_bigquery_current_time(current_time_df, time_schema)

    # df.to_csv("datetime.csv", index = False)

    return df

def load_to_bigquery(df):

    df.to_csv("test1.csv", index = False)
    print("Inside Bigquery ")
    table_id = "hnmc-data-science.wheniwork.listShifts"

    # Specifying all table schema
    # with open('/listShift.json') as json_schema: 
    #     file_schema = json.load(json_schema)
        
    print("Inside schema")
    #print(file_schema['listShifts'])
    job_config = bigquery.LoadJobConfig(
        
      schema = listShift_schema,  
      write_disposition="WRITE_APPEND"
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    )

    job = client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

def load_to_bigquery_current_time(df, time_schema):

    print("Inside Bigquery ")
    project_id = 'hnmc-data-science'
    dataset_id = 'wheniwork'

    table_id = f"{project_id}.{dataset_id}.currentShiftTime"

    # Specifying all table schema
    # with open('/listShift.json') as json_schema: 
    #     file_schema = json.load(json_schema)
    # print(type(file_schema[filename]))  
    print("Inside schema")
   
    #print(file_schema['listShifts'])
    job_config = bigquery.LoadJobConfig(
          
      write_disposition="WRITE_TRUNCATE", schema = time_schema
      #, encoding = 'ISO-8859-1'
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    )

    job = client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )
  
main()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



datetime64[ns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4634 entries, 0 to 4633
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   id                             4634 non-null   int64         
 1   account_id                     4634 non-null   int64         
 2   user_id                        4634 non-null   int64         
 3   location_id                    4634 non-null   int64         
 4   position_id                    4634 non-null   int64         
 5   site_id                        4634 non-null   int64         
 6   start_time                     4634 non-null   datetime64[ns]
 7   end_time                       4634 non-null   datetime64[ns]
 8   break_time                     4634 non-null   float64       
 9   color                          4634 non-null   object        
 10  notes                          4634 non-null   object        
 11  al

ArrowTypeError: ignored

In [ ]:
# Pushing 1st range data from 1st Nov 2020 to 31st jan 2020 final code  ---- second version of code while working with Updated_at >  current time

import requests
import json
from datetime import datetime
from datetime import timedelta
import os
import hashlib
import datetime as dt
import pandas as pd
from copy import deepcopy
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
from pandas import DataFrame

project_id = 'hnmc-data-science'
dataset_id = 'wheniwork'
table_id = 'listShifts'

# TODO(developer): Set key_path to the path to the service account key
#                  file.

key_path = "/hnmc-data-science-1e5996a1f37c.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(credentials=credentials, project=project_id)
dataset = client.dataset(dataset_id)
table_ref = dataset.table(table_id)

time_schema = [
    bigquery.SchemaField("current_time", "TIMESTAMP", mode="REQUIRED")]

listShift_schema = [
       bigquery.SchemaField("id", "INT64", mode="REQUIRED"),
       bigquery.SchemaField("account_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("user_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("location_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("position_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("site_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("start_time", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("end_time", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("break_time", "FLOAT64", mode="NULLABLE"),
       bigquery.SchemaField("color", "STRING", mode="NULLABLE"),
       bigquery.SchemaField("notes", "STRING", mode="NULLABLE"),
       bigquery.SchemaField("alerted", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("linked_users", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("shiftchain_key", "STRING", mode="NULLABLE"),
       bigquery.SchemaField("published", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("published_date", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("notified_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("instances", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("created_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("updated_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("acknowledged", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("acknowledged_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("creator_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("is_open", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("actionable", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("block_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("requires_openshift_approval", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("openshift_approval_request_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("is_shared", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("UNIQUE_KEY", "STRING", mode="REQUIRED"),
       bigquery.SchemaField("VERSION_DATE", "TIMESTAMP", mode="NULLABLE")
       ]

def main():

  #  df = get_all_data()
  #  load_to_bigquery(df)

   last_query_time_df = fetch_data()
   get_all_data(last_query_time_df)

def fetch_data():

    sql_source = """ SELECT * FROM `hnmc-data-science.wheniwork.currentShiftTime` """             

    df_source = client.query(sql_source).to_dataframe()
    df_source['current_time'] = df_source['current_time'].astype(str).str[:-6]
    df_source['current_time'] = pd.to_datetime(df_source['current_time'], format='%d %b %Y %H:%M:%S', errors='ignore')

    return df_source

def get_all_data(last_query_time_df):

    r = requests.post('https://api.login.wheniwork.com/login', json={"email":"rdoshi@holyname.org","password":"wheniwork123"})
    r.status_code
    data= r.json()

    token=data["token"]
    #print(token)
    endpoint = "https://api.wheniwork.com/2/shifts"

    headers = {"Authorization": token}

    all_data = requests.get(endpoint, params={'start': '01 Nov 2020', 'end': '31 Jan 2021', 'include_allopen': True}, headers=headers).json()
    print(all_data)

    id_ofall_data = []
    for i in range(0, len(all_data['shifts'])):

      id_ofall_data.append(all_data['shifts'][i]['id'])

    print(id_ofall_data)

    new_list = []
    for i in range(0, len(all_data['shifts'])):

      if all_data['shifts'][i]['id'] in id_ofall_data: 
        
        #print(all_data['shifts'][i])
        new_list.append(all_data['shifts'][i])

    # convert list of dictionary to dataframe

    #print(new_list)
    df = pd.DataFrame(new_list)

    df.to_csv("before_time_conv.csv", index = False)
    df['start_time'] = df['start_time'].astype(str).str[5:-6]
    df['end_time'] = df['end_time'].astype(str).str[5:-6]
    df['published_date'] = df['published_date'].astype(str).str[5:-6]
    df['notified_at'] = df['notified_at'].astype(str).str[5:-6]
    df['created_at'] = df['created_at'].astype(str).str[5:-6]
    df['updated_at'] = df['updated_at'].astype(str).str[5:-6]
    df['acknowledged_at'] = df['acknowledged_at'].astype(str).str[5:-6]
    df.to_csv("after_time_conv.csv", index = False)
    # print(df)
    df['start_time'] =  pd.to_datetime(df['start_time'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['end_time'] =  pd.to_datetime(df['end_time'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['published_date'] =  pd.to_datetime(df['published_date'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['notified_at'] =  pd.to_datetime(df['notified_at'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['created_at'] =  pd.to_datetime(df['created_at'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['updated_at'] =  pd.to_datetime(df['updated_at'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['acknowledged_at'] =  pd.to_datetime(df['acknowledged_at'], format='%d %b %Y %H:%M:%S', errors='ignore')
   
    print(df['updated_at'].dtypes)
    #generating Hash key
    gene_hash = {
        'listShifts': ['id']
    }

    key_combination = gene_hash.get('listShifts')
    
    df['UNIQUE_KEY'] = df[key_combination].apply(lambda row: '_'.join(row.values.astype(
        str)), axis=1).astype(str).str.encode('utf-8').apply(lambda x: (hashlib.sha512(x).hexdigest().upper()))

    df['VERSION_DATE'] = datetime.now()

    # check for updated data from API call

    print(df['updated_at'])
    print(last_query_time_df['current_time'][0])
    # last_query_time_df['current_time'] = last_query_time_df['current_time'].astype(str).str[:-6]
    # last_query_time_df['current_time'] =  pd.to_datetime(last_query_time_df['current_time'], format='%d %b %Y %H:%M:%S', errors='ignore')
    print(last_query_time_df['current_time'][0])
    new_data_df = df[df['updated_at'] >= (last_query_time_df['current_time'][0])]
    print(new_data_df.info())
    load_to_bigquery(new_data_df, listShift_schema)

    # Assign current time to our new currentShiftTime table

    current_shift_time = datetime.now() - timedelta(hours=5)
    current_time = {'current_time': current_shift_time.strftime("%Y-%m-%d %H:%M:%S")}
    
    current_time_df = pd.DataFrame(current_time, columns = ['current_time'], index=[0])

    current_time_df['current_time'] =  pd.to_datetime(current_time_df['current_time'], format='%Y-%m-%d %H:%M:%S')

    load_to_bigquery_current_time(current_time_df, time_schema)

    # df.to_csv("datetime.csv", index = False)

   # return df

def load_to_bigquery(df, listShift_schema):

    df.to_csv("test1.csv", index = False)
    print("Inside Bigquery ")
    table_id = "hnmc-data-science.wheniwork.listShifts"

    table = client.get_table(table_id)  # Make an API request.
    print(
        "before Loading new data {} rows and {} columns are present in {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )
    # Specifying all table schema
    # with open('/listShift.json') as json_schema: 
    #     file_schema = json.load(json_schema)
        
    print("Inside schema")
    #print(file_schema['listShifts'])
    job_config = bigquery.LoadJobConfig(
        
      #schema = listShift_schema,  
      write_disposition="WRITE_APPEND"
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    )

    job = client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    table = client.get_table(table_id)  # Make an API request.
    print(
        "After Loading new data {} rows and {} columns are present in {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

def load_to_bigquery_current_time(df, time_schema):

    print("Inside Bigquery ")
    project_id = 'hnmc-data-science'
    dataset_id = 'wheniwork'

    table_id = f"{project_id}.{dataset_id}.currentShiftTime"

    # Specifying all table schema
    # with open('/listShift.json') as json_schema: 
    #     file_schema = json.load(json_schema)
    # print(type(file_schema[filename]))  
    print("Inside current time schema")
   
    #print(file_schema['listShifts'])
    job_config = bigquery.LoadJobConfig(
          
      write_disposition="WRITE_TRUNCATE", schema = time_schema
      #, encoding = 'ISO-8859-1'
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    )

    job = client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )
  
main()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



datetime64[ns]
0      2020-10-31 09:00:39
1      2020-10-31 13:00:41
2      2020-10-31 17:00:24
3      2020-10-31 17:00:23
4      2020-10-31 17:00:23
               ...        
3957   2020-12-03 13:34:17
3958   2020-12-03 13:34:17
3959   2020-12-03 13:34:17
3960   2020-12-03 13:34:17
3961   2020-12-03 13:34:17
Name: updated_at, Length: 3962, dtype: datetime64[ns]
2020-12-11 15:09:19
2020-12-11 15:09:19
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   id                             0 non-null      int64         
 1   account_id                     0 non-null      int64         
 2   user_id                        0 non-null      int64         
 3   location_id                    0 non-null      int64         
 4   position_id                    0 non-null      int64         
 5   site_id                    

In [37]:
# Pushing 1st range data from 1st Nov 2020 to 31st jan 2020 final code  ---- third version of code while working with Updated_at >  current time

import requests
import json
from datetime import datetime
from datetime import timedelta
import os
import hashlib
import datetime as dt
import pandas as pd
from copy import deepcopy
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
from pandas import DataFrame

project_id = 'hnmc-data-science'
dataset_id = 'wheniwork'
table_id = 'listShifts'

# TODO(developer): Set key_path to the path to the service account key
#                  file.

key_path = "/hnmc-data-science-1e5996a1f37c.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(credentials=credentials, project=project_id)
dataset = client.dataset(dataset_id)
table_ref = dataset.table(table_id)

time_schema = [
    bigquery.SchemaField("current_time", "TIMESTAMP", mode="REQUIRED")]

listShift_schema = [
       bigquery.SchemaField("id", "INT64", mode="REQUIRED"),
       bigquery.SchemaField("account_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("user_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("location_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("position_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("site_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("start_time", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("end_time", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("break_time", "FLOAT64", mode="NULLABLE"),
       bigquery.SchemaField("color", "STRING", mode="NULLABLE"),
       bigquery.SchemaField("notes", "STRING", mode="NULLABLE"),
       bigquery.SchemaField("alerted", "BOOL", mode="NULLABLE"),
      # bigquery.SchemaField("linked_users", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("shiftchain_key", "STRING", mode="NULLABLE"),
       bigquery.SchemaField("published", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("published_date", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("notified_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("instances", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("created_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("updated_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("acknowledged", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("acknowledged_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("creator_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("is_open", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("actionable", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("block_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("requires_openshift_approval", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("openshift_approval_request_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("is_shared", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("UNIQUE_KEY", "STRING", mode="REQUIRED"),
       bigquery.SchemaField("VERSION_DATE", "TIMESTAMP", mode="NULLABLE")
       ]

def main():

  #  df = get_all_data()
  #  load_to_bigquery(df)

   last_query_time_df = fetch_data()
   get_all_data(last_query_time_df)

def fetch_data():

    sql_source = """ SELECT * FROM `hnmc-data-science.wheniwork.currentShiftTime` """             

    df_source = client.query(sql_source).to_dataframe()
    df_source['current_time'] = df_source['current_time'].astype(str).str[:-6]
    df_source['current_time'] = pd.to_datetime(df_source['current_time'], format='%d %b %Y %H:%M:%S', errors='ignore')

    return df_source

def get_all_data(last_query_time_df):

    r = requests.post('https://api.login.wheniwork.com/login', json={"email":"rdoshi@holyname.org","password":"wheniwork123"})
    r.status_code
    data= r.json()

    token=data["token"]
    #print(token)

    f = open('/wiw_listShifts.json',) 
    data = json.load(f)

    for key, value in data.items():

      endpoint = f"https://api.wheniwork.com/2/{data[key]['api_endpoint_name']}"
      print(endpoint)
      headers = {"Authorization": token}

      all_data = requests.get(endpoint, params={'start': '01 Nov 2020', 'end': '31 Jan 2021', 'include_allopen': True}, headers=headers).json()
      print(all_data)
      new_list = all_data[data[key]['api_endpoint_name']]
      #print(new_list)

      final_dataframe = []
      for i in range(len(new_list)):

        dictionary = {x: new_list[i][x] for x in data[key]['columns']}
        final_dataframe.append(dictionary)

      #print(final_dataframe)

      # convert list of dictionary to dataframe
      df = pd.DataFrame(final_dataframe)

#     id_ofall_data = []
#     for i in range(0, len(all_data['shifts'])):

#       id_ofall_data.append(all_data['shifts'][i]['id'])

#     print(id_ofall_data)

#     new_list = []
#     for i in range(0, len(all_data['shifts'])):

#       if all_data['shifts'][i]['id'] in id_ofall_data: 
        
#         #print(all_data['shifts'][i])
#         new_list.append(all_data['shifts'][i])

#     # convert list of dictionary to dataframe

#     #print(new_list)
#     df = pd.DataFrame(new_list)

    df.to_csv("before_time_conv.csv", index = False)
    df['start_time'] = df['start_time'].astype(str).str[5:-6]
    df['end_time'] = df['end_time'].astype(str).str[5:-6]
    df['published_date'] = df['published_date'].astype(str).str[5:-6]
    df['notified_at'] = df['notified_at'].astype(str).str[5:-6]
    df['created_at'] = df['created_at'].astype(str).str[5:-6]
    df['updated_at'] = df['updated_at'].astype(str).str[5:-6]
    df['acknowledged_at'] = df['acknowledged_at'].astype(str).str[5:-6]
    df.to_csv("after_time_conv.csv", index = False)
    # print(df)
    df['start_time'] =  pd.to_datetime(df['start_time'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['end_time'] =  pd.to_datetime(df['end_time'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['published_date'] =  pd.to_datetime(df['published_date'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['notified_at'] =  pd.to_datetime(df['notified_at'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['created_at'] =  pd.to_datetime(df['created_at'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['updated_at'] =  pd.to_datetime(df['updated_at'], format='%d %b %Y %H:%M:%S', errors='ignore')
    df['acknowledged_at'] =  pd.to_datetime(df['acknowledged_at'], format='%d %b %Y %H:%M:%S', errors='ignore')
   
    print(df['updated_at'].dtypes)
    #generating Hash key
    gene_hash = {
        'listShifts': ['id']
    }

    key_combination = gene_hash.get('listShifts')
    
    df['UNIQUE_KEY'] = df[key_combination].apply(lambda row: '_'.join(row.values.astype(
        str)), axis=1).astype(str).str.encode('utf-8').apply(lambda x: (hashlib.sha512(x).hexdigest().upper()))

    df['VERSION_DATE'] = datetime.now()

    # check for updated data from API call

    print(df['updated_at'])
    print(last_query_time_df['current_time'][0])
    last_query_time_df['current_time'] = last_query_time_df['current_time'].astype(str).str[:-6]
    last_query_time_df['current_time'] =  pd.to_datetime(last_query_time_df['current_time'], format='%d %b %Y %H:%M:%S', errors='ignore')
    print(last_query_time_df['current_time'][0])
    new_data_df = df[df['updated_at'] >= (last_query_time_df['current_time'][0])]
    print(new_data_df.info())
    load_to_bigquery(new_data_df, listShift_schema)

    # Assign current time to our new currentShiftTime table

    current_shift_time = datetime.now() - timedelta(hours=5)
    current_time = {'current_time': current_shift_time.strftime("%Y-%m-%d %H:%M:%S")}
    
    current_time_df = pd.DataFrame(current_time, columns = ['current_time'], index=[0])

    current_time_df['current_time'] =  pd.to_datetime(current_time_df['current_time'], format='%Y-%m-%d %H:%M:%S')

    load_to_bigquery_current_time(current_time_df, time_schema)

#     # df.to_csv("datetime.csv", index = False)

#    # return df

def load_to_bigquery(df, listShift_schema):

    df.to_csv("test1.csv", index = False)
    print("Inside Bigquery ")
    table_id = "hnmc-data-science.wheniwork.listShifts"

    table = client.get_table(table_id)  # Make an API request.
    print(
        "before Loading new data {} rows and {} columns are present in {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )
    # Specifying all table schema
    # with open('/listShift.json') as json_schema: 
    #     file_schema = json.load(json_schema)
        
    print("Inside schema")
    #print(file_schema['listShifts'])
    job_config = bigquery.LoadJobConfig(
        
      #schema = listShift_schema,  
      write_disposition="WRITE_APPEND"
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    )

    job = client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    table = client.get_table(table_id)  # Make an API request.
    print(
        "After Loading new data {} rows and {} columns are present in {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

def load_to_bigquery_current_time(df, time_schema):

    print("Inside Bigquery ")
    project_id = 'hnmc-data-science'
    dataset_id = 'wheniwork'

    table_id = f"{project_id}.{dataset_id}.currentShiftTime"

    # Specifying all table schema
    # with open('/listShift.json') as json_schema: 
    #     file_schema = json.load(json_schema)
    # print(type(file_schema[filename]))  
    print("Inside current time schema")
   
    #print(file_schema['listShifts'])
    job_config = bigquery.LoadJobConfig(
          
      write_disposition="WRITE_TRUNCATE", schema = time_schema
      #, encoding = 'ISO-8859-1'
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    )

    job = client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )
  
main()

https://api.wheniwork.com/2/shifts


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



datetime64[ns]
0      2020-10-31 09:00:39
1      2020-10-31 13:00:41
2      2020-10-31 17:00:24
3      2020-10-31 17:00:23
4      2020-10-31 17:00:23
               ...        
4629   2020-12-03 13:34:17
4630   2020-12-03 13:34:17
4631   2020-12-03 13:34:17
4632   2020-12-03 13:34:17
4633   2020-12-03 13:34:17
Name: updated_at, Length: 4634, dtype: datetime64[ns]
2020-12-11 16:03:34
2020-12-11 16
<class 'pandas.core.frame.DataFrame'>
Int64Index: 695 entries, 2420 to 4578
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   id                             695 non-null    int64         
 1   account_id                     695 non-null    int64         
 2   user_id                        695 non-null    int64         
 3   location_id                    695 non-null    int64         
 4   position_id                    695 non-null    int64         
 5   site_id          

In [ ]:
# Pushing 1st range data from 1st Nov 2020 to 31st jan 2020 ==== final code 11th Dec 2020 10 PM 

import requests
import json
from datetime import datetime
from datetime import timedelta
import os
import hashlib
import datetime as dt
import pandas as pd
from copy import deepcopy
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
from pandas import DataFrame

project_id = 'hnmc-data-science'
dataset_id = 'wheniwork'
table_id = 'listShifts'

# TODO(developer): Set key_path to the path to the service account key
#                  file.

key_path = "/hnmc-data-science-1e5996a1f37c.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(credentials=credentials, project=project_id)
dataset = client.dataset(dataset_id)
table_ref = dataset.table(table_id)
time_schema = [
    bigquery.SchemaField("current_time", "TIMESTAMP", mode="REQUIRED")]

def main():

    df = get_all_data()
    load_to_bigquery(df)

def get_all_data():

    r = requests.post('https://api.login.wheniwork.com/login', json={"email":"rdoshi@holyname.org","password":"wheniwork123"})
    r.status_code
    data= r.json()

    token=data["token"]
    #print(token)
    endpoint = "https://api.wheniwork.com/2/shifts"

    headers = {"Authorization": token}

    all_data = requests.get(endpoint, params={'start': '01 Nov 2020', 'end': '31 Jan 2021', 'include_allopen': True}, headers=headers).json()
    print(all_data)

    id_ofall_data = []
    for i in range(0, len(all_data['shifts'])):

      id_ofall_data.append(all_data['shifts'][i]['id'])

    print(id_ofall_data)

    new_list = []
    for i in range(0, len(all_data['shifts'])):

      if all_data['shifts'][i]['id'] in id_ofall_data: 
        
        #print(all_data['shifts'][i])
        new_list.append(all_data['shifts'][i])

    # convert list of dictionary to dataframe

    #print(new_list)
    df = pd.DataFrame(new_list)
    # print(df)
    df['start_time'] =  pd.to_datetime(df['start_time'], format='%a, %d %b %Y %H:%M:%S %z', errors='ignore')
    df['end_time'] =  pd.to_datetime(df['end_time'], format='%a, %d %b %Y %H:%M:%S %z', errors='ignore')
    df['published_date'] =  pd.to_datetime(df['published_date'], format='%a, %d %b %Y %H:%M:%S %z', errors='ignore')
    df['notified_at'] =  pd.to_datetime(df['notified_at'], format='%a, %d %b %Y %H:%M:%S %z', errors='ignore')
    df['created_at'] =  pd.to_datetime(df['created_at'], format='%a, %d %b %Y %H:%M:%S %z', errors='ignore')
    df['updated_at'] =  pd.to_datetime(df['updated_at'], format='%a, %d %b %Y %H:%M:%S %z', errors='ignore')
    df['acknowledged_at'] =  pd.to_datetime(df['acknowledged_at'], format='%a, %d %b %Y %H:%M:%S %z', errors='ignore')
    print(df['updated_at'].dtypes)
    print(df['updated_at'])
    df['updated_at'] = df['updated_at'].astype(str).str[:-6]
    df['updated_at'] = pd.to_datetime(df['updated_at'])
    print(df['updated_at'].dtypes)
    print(df['updated_at'])
    #generating Hash key
    gene_hash = {
        'listShifts': ['id']
    }

    key_combination = gene_hash.get('listShifts')
    
    df['UNIQUE_KEY'] = df[key_combination].apply(lambda row: '_'.join(row.values.astype(
        str)), axis=1).astype(str).str.encode('utf-8').apply(lambda x: (hashlib.sha512(x).hexdigest().upper()))

    df['VERSION_DATE'] = datetime.now()
    df.drop('linked_users', axis=1, inplace=True)

    df.to_csv("test.csv", index = False)

    current_shift_time = datetime.now() - timedelta(hours=5)
    current_time = {'current_time': current_shift_time.strftime("%Y-%m-%d %H:%M:%S")}
    
    current_time_df = pd.DataFrame(current_time, columns = ['current_time'], index=[0])

    current_time_df['current_time'] =  pd.to_datetime(current_time_df['current_time'], format='%Y-%m-%d %H:%M:%S')

    load_to_bigquery_current_time(current_time_df, time_schema)

    return df

def load_to_bigquery(df):

    print("Inside Bigquery ")
    table_id = "hnmc-data-science.wheniwork.listShifts"

    # Specifying all table schema
    # with open('/listShift.json') as json_schema: 
    #     file_schema = json.load(json_schema)
        
    print("Inside schema")
    #print(file_schema['listShifts'])
    job_config = bigquery.LoadJobConfig(
        
      schema = [
       bigquery.SchemaField("id", "INT64", mode="REQUIRED"),
       bigquery.SchemaField("account_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("user_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("location_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("position_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("site_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("start_time", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("end_time", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("break_time", "FLOAT64", mode="NULLABLE"),
       bigquery.SchemaField("color", "STRING", mode="NULLABLE"),
       bigquery.SchemaField("notes", "STRING", mode="NULLABLE"),
       bigquery.SchemaField("alerted", "BOOL", mode="NULLABLE"),
       #bigquery.SchemaField("linked_users", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("shiftchain_key", "STRING", mode="NULLABLE"),
       bigquery.SchemaField("published", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("published_date", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("notified_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("instances", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("created_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("updated_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("acknowledged", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("acknowledged_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("creator_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("is_open", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("actionable", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("block_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("requires_openshift_approval", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("openshift_approval_request_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("is_shared", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("UNIQUE_KEY", "STRING", mode="REQUIRED"),
       bigquery.SchemaField("VERSION_DATE", "TIMESTAMP", mode="NULLABLE")
       ],  
      write_disposition="WRITE_APPEND"
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    )

    job = client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

def load_to_bigquery_current_time(df, time_schema):

    print("Inside Bigquery ")
    project_id = 'hnmc-data-science'
    dataset_id = 'wheniwork'

    table_id = f"{project_id}.{dataset_id}.currentShiftTime"

    # Specifying all table schema
    # with open('/listShift.json') as json_schema: 
    #     file_schema = json.load(json_schema)
    # print(type(file_schema[filename]))  
    print("Inside schema")
   
    #print(file_schema['listShifts'])
    job_config = bigquery.LoadJobConfig(
          
      write_disposition="WRITE_TRUNCATE", schema = time_schema
      #, encoding = 'ISO-8859-1'
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    )

    job = client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )
        
main()

In [50]:
# Pushing 1st range data from 1st Nov 2020 to 31st jan 2020 final code  ---- 
# second version of code while working with Updated_at >  current time 11th DEC 2020 10:20 PM

import requests
import json
from datetime import datetime
from datetime import timedelta
import os
import hashlib
import datetime as dt
import pandas as pd
from copy import deepcopy
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
from pandas import DataFrame

project_id = 'hnmc-data-science'
dataset_id = 'wheniwork'
table_id = 'listShifts'

# TODO(developer): Set key_path to the path to the service account key
#                  file.

key_path = "/hnmc-data-science-1e5996a1f37c.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(credentials=credentials, project=project_id)
dataset = client.dataset(dataset_id)
table_ref = dataset.table(table_id)

time_schema = [
    bigquery.SchemaField("current_time", "TIMESTAMP", mode="REQUIRED")]

listShift_schema = [
       bigquery.SchemaField("id", "INT64", mode="REQUIRED"),
       bigquery.SchemaField("account_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("user_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("location_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("position_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("site_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("start_time", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("end_time", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("break_time", "FLOAT64", mode="NULLABLE"),
       bigquery.SchemaField("color", "STRING", mode="NULLABLE"),
       bigquery.SchemaField("notes", "STRING", mode="NULLABLE"),
       bigquery.SchemaField("alerted", "BOOL", mode="NULLABLE"),
       #bigquery.SchemaField("linked_users", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("shiftchain_key", "STRING", mode="NULLABLE"),
       bigquery.SchemaField("published", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("published_date", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("notified_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("instances", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("created_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("updated_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("acknowledged", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("acknowledged_at", "TIMESTAMP", mode="NULLABLE"),
       bigquery.SchemaField("creator_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("is_open", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("actionable", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("block_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("requires_openshift_approval", "BOOL", mode="NULLABLE"),
       bigquery.SchemaField("openshift_approval_request_id", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("is_shared", "INT64", mode="NULLABLE"),
       bigquery.SchemaField("UNIQUE_KEY", "STRING", mode="REQUIRED"),
       bigquery.SchemaField("VERSION_DATE", "TIMESTAMP", mode="NULLABLE")
       ]

def main():

  #  df = get_all_data()
  #  load_to_bigquery(df)

   last_query_time_df = fetch_data()
   get_all_data(last_query_time_df)

def fetch_data():

    sql_source = """ SELECT * FROM `hnmc-data-science.wheniwork.currentShiftTime` """             

    df_source = client.query(sql_source).to_dataframe()
    df_source['current_time'] = df_source['current_time'].astype(str).str[:-6]
    df_source['current_time'] = pd.to_datetime(df_source['current_time'], format='%d %b %Y %H:%M:%S', errors='ignore')

    return df_source

def get_all_data(last_query_time_df):

    r = requests.post('https://api.login.wheniwork.com/login', json={"email":"rdoshi@holyname.org","password":"wheniwork123"})
    r.status_code
    data= r.json()

    token=data["token"]
    #print(token)
    endpoint = "https://api.wheniwork.com/2/shifts"

    headers = {"Authorization": token}

    all_data = requests.get(endpoint, params={'start': '01 Nov 2020', 'end': '31 Jan 2021', 'include_allopen': True}, headers=headers).json()
    print(all_data)

    id_ofall_data = []
    for i in range(0, len(all_data['shifts'])):

      id_ofall_data.append(all_data['shifts'][i]['id'])

    print(id_ofall_data)

    new_list = []
    for i in range(0, len(all_data['shifts'])):

      if all_data['shifts'][i]['id'] in id_ofall_data: 
        
        #print(all_data['shifts'][i])
        new_list.append(all_data['shifts'][i])

    # convert list of dictionary to dataframe

    #print(new_list)
    df = pd.DataFrame(new_list)

    # df.to_csv("before_time_conv.csv", index = False)
    # df['start_time'] = df['start_time'].astype(str).str[5:-6]
    # df['end_time'] = df['end_time'].astype(str).str[5:-6]
    # df['published_date'] = df['published_date'].astype(str).str[5:-6]
    # df['notified_at'] = df['notified_at'].astype(str).str[5:-6]
    # df['created_at'] = df['created_at'].astype(str).str[5:-6]
    # df['updated_at'] = df['updated_at'].astype(str).str[5:-6]
    # df['acknowledged_at'] = df['acknowledged_at'].astype(str).str[5:-6]
    # df.to_csv("after_time_conv.csv", index = False)
    # # print(df)
    # df['start_time'] =  pd.to_datetime(df['start_time'], format='%d %b %Y %H:%M:%S', errors='ignore')
    # df['end_time'] =  pd.to_datetime(df['end_time'], format='%d %b %Y %H:%M:%S', errors='ignore')
    # df['published_date'] =  pd.to_datetime(df['published_date'], format='%d %b %Y %H:%M:%S', errors='ignore')
    # df['notified_at'] =  pd.to_datetime(df['notified_at'], format='%d %b %Y %H:%M:%S', errors='ignore')
    # df['created_at'] =  pd.to_datetime(df['created_at'], format='%d %b %Y %H:%M:%S', errors='ignore')
    # df['updated_at'] =  pd.to_datetime(df['updated_at'], format='%d %b %Y %H:%M:%S', errors='ignore')
    # df['acknowledged_at'] =  pd.to_datetime(df['acknowledged_at'], format='%d %b %Y %H:%M:%S', errors='ignore')

    df['start_time'] =  pd.to_datetime(df['start_time'], format='%a, %d %b %Y %H:%M:%S %z', errors='ignore')
    df['end_time'] =  pd.to_datetime(df['end_time'], format='%a, %d %b %Y %H:%M:%S %z', errors='ignore')
    df['published_date'] =  pd.to_datetime(df['published_date'], format='%a, %d %b %Y %H:%M:%S %z', errors='ignore')
    df['notified_at'] =  pd.to_datetime(df['notified_at'], format='%a, %d %b %Y %H:%M:%S %z', errors='ignore')
    df['created_at'] =  pd.to_datetime(df['created_at'], format='%a, %d %b %Y %H:%M:%S %z', errors='ignore')
    df['updated_at'] =  pd.to_datetime(df['updated_at'], format='%a, %d %b %Y %H:%M:%S %z', errors='ignore')
    df['acknowledged_at'] =  pd.to_datetime(df['acknowledged_at'], format='%a, %d %b %Y %H:%M:%S %z', errors='ignore')
    print(df['updated_at'].dtypes)
    print(df['updated_at'])
    df['updated_at'] = df['updated_at'].astype(str).str[:-6]
    df['updated_at'] = pd.to_datetime(df['updated_at'])
    print(df['updated_at'].dtypes)

    #generating Hash key
    gene_hash = {
        'listShifts': ['id']
    }

    key_combination = gene_hash.get('listShifts')
    
    df['UNIQUE_KEY'] = df[key_combination].apply(lambda row: '_'.join(row.values.astype(
        str)), axis=1).astype(str).str.encode('utf-8').apply(lambda x: (hashlib.sha512(x).hexdigest().upper()))

    df['VERSION_DATE'] = datetime.now()
    df.drop('linked_users', axis=1, inplace=True)


    # check for updated data from API call

    print(df['updated_at'])
    print(last_query_time_df['current_time'][0])
    # last_query_time_df['current_time'] = last_query_time_df['current_time'].astype(str).str[:-6]
    # last_query_time_df['current_time'] =  pd.to_datetime(last_query_time_df['current_time'], format='%d %b %Y %H:%M:%S', errors='ignore')
    print(last_query_time_df['current_time'][0])
    new_data_df = df[df['updated_at'] >= (last_query_time_df['current_time'][0])]
    print(new_data_df.info())
    load_to_bigquery(new_data_df, listShift_schema)

    # Assign current time to our new currentShiftTime table

    current_shift_time = datetime.now() - timedelta(hours=5)
    current_time = {'current_time': current_shift_time.strftime("%Y-%m-%d %H:%M:%S")}
    
    current_time_df = pd.DataFrame(current_time, columns = ['current_time'], index=[0])

    current_time_df['current_time'] =  pd.to_datetime(current_time_df['current_time'], format='%Y-%m-%d %H:%M:%S')

    load_to_bigquery_current_time(current_time_df, time_schema)

    # df.to_csv("datetime.csv", index = False)

   # return df

def load_to_bigquery(df, listShift_schema):

    df.to_csv("test1.csv", index = False)
    print("Inside Bigquery ")
    table_id = "hnmc-data-science.wheniwork.listShifts"

    table = client.get_table(table_id)  # Make an API request.
    print(
        "before Loading new data {} rows and {} columns are present in {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )
    # Specifying all table schema
    # with open('/listShift.json') as json_schema: 
    #     file_schema = json.load(json_schema)
        
    print("Inside schema")
    #print(file_schema['listShifts'])
    job_config = bigquery.LoadJobConfig(
        
      #schema = listShift_schema,  
      write_disposition="WRITE_APPEND"
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    )

    job = client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    table = client.get_table(table_id)  # Make an API request.
    print(
        "After Loading new data {} rows and {} columns are present in {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

def load_to_bigquery_current_time(df, time_schema):

    print("Inside Bigquery ")
    project_id = 'hnmc-data-science'
    dataset_id = 'wheniwork'

    table_id = f"{project_id}.{dataset_id}.currentShiftTime"

    # Specifying all table schema
    # with open('/listShift.json') as json_schema: 
    #     file_schema = json.load(json_schema)
    # print(type(file_schema[filename]))  
    print("Inside current time schema")
   
    #print(file_schema['listShifts'])
    job_config = bigquery.LoadJobConfig(
          
      write_disposition="WRITE_TRUNCATE", schema = time_schema
      #, encoding = 'ISO-8859-1'
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    )

    job = client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )
  
main()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



object
0       2020-10-31 09:00:39-04:00
1       2020-10-31 13:00:41-04:00
2       2020-10-31 17:00:24-04:00
3       2020-10-31 17:00:23-04:00
4       2020-10-31 17:00:23-04:00
                  ...            
4629    2020-12-03 13:34:17-05:00
4630    2020-12-03 13:34:17-05:00
4631    2020-12-03 13:34:17-05:00
4632    2020-12-03 13:34:17-05:00
4633    2020-12-03 13:34:17-05:00
Name: updated_at, Length: 4634, dtype: object
datetime64[ns]
0      2020-10-31 09:00:39
1      2020-10-31 13:00:41
2      2020-10-31 17:00:24
3      2020-10-31 17:00:23
4      2020-10-31 17:00:23
               ...        
4629   2020-12-03 13:34:17
4630   2020-12-03 13:34:17
4631   2020-12-03 13:34:17
4632   2020-12-03 13:34:17
4633   2020-12-03 13:34:17
Name: updated_at, Length: 4634, dtype: datetime64[ns]
2020-12-11 22:00:20
2020-12-11 22:00:20
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype                 